In [22]:
import unittest
import numpy

import sys
sys.path.append('../')

from src.basis_class import Basis


In [23]:
x = Basis(3).dimension()

RecursionError: maximum recursion depth exceeded

In [12]:
class BasisTestCase(unittest.TestCase):
    
    def test_get_correct_dimensions(self):
        n = 2
        basis_class = Basis(2)
        result = basis_class.dimension
        self.assertEquals(result, n)

        

In [13]:
unittest.main(argv=[''], exit=False)

E
ERROR: test_get_correct_dimensions (__main__.BasisTestCase.test_get_correct_dimensions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\waqas\AppData\Local\Temp\ipykernel_11692\4248421554.py", line 5, in test_get_correct_dimensions
    result = basis_class.dimension
             ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\waqas\OneDrive\Documents\CVUT\Survey\Code\Lie Algebra\Generating_Structure_Constants\tests\..\src\basis_class.py", line 10, in dimension
    return self.dimension
           ^^^^^^^^^^^^^^
  File "c:\Users\waqas\OneDrive\Documents\CVUT\Survey\Code\Lie Algebra\Generating_Structure_Constants\tests\..\src\basis_class.py", line 10, in dimension
    return self.dimension
           ^^^^^^^^^^^^^^
  File "c:\Users\waqas\OneDrive\Documents\CVUT\Survey\Code\Lie Algebra\Generating_Structure_Constants\tests\..\src\basis_class.py", line 10, in dimension
    return self.dimension
           ^^^^^^^^^^^^^^
  [